In [ ]:
import pandas, json, numpy, requests, os, datetime, statistics, math, pytz, tweepy, sqlite3, time, re, random, matplotlib.pyplot as plt, sklearn, statsmodels.api as sm
from bs4 import BeautifulSoup
from scipy.signal import find_peaks

This script is for determining the earnings score

In [ ]:
def earnings_score(symbol, name):
    try:
        print(name)
        earnings_summary_df = pandas.read_csv(os.getcwd() + '\\Earnings Summary\\' + symbol + ' - '+ name +' Earnings Summary - 2016-2020.csv')

        q1_df = pandas.read_csv(os.getcwd() + '\\Earnings Dates\\Earnings - Quarter 1 - 2016-2020.csv')
        q2_df = pandas.read_csv(os.getcwd() + '\\Earnings Dates\\Earnings - Quarter 2 - 2016-2020.csv')
        q3_df = pandas.read_csv(os.getcwd() + '\\Earnings Dates\\Earnings - Quarter 3 - 2016-2020.csv')
        q4_df = pandas.read_csv(os.getcwd() + '\\Earnings Dates\\Earnings - Quarter 4 - 2016-2020.csv')

        overall_average_surprise = numpy.nanmean(earnings_summary_df.Average_Surprise)

        combined_df = pandas.DataFrame()

        for x in [q1_df, q2_df, q3_df,q4_df]:
            temp_df = x[x.Symbol == symbol]

            combined_df = pandas.concat([combined_df, temp_df])

        combined_df['Surprise_Overall_Average'] = overall_average_surprise

        combined_df = combined_df.reset_index()

        for y in range(len(combined_df)):
            earnings_score = 0

            if combined_df.loc[y, 'EPS_Estimate'] > 0:
                earnings_score = 500

            elif combined_df.loc[y, 'EPS_Estimate'] < 0:
                earnings_score = -500

            elif math.isnan(combined_df.loc[y, 'EPS_Estimate']) == True:
                try:
                    if combined_df.loc[y-1, 'EPS_Estimate'] != numpy.NaN:
                        earnings_score = combined_df.loc[y-1, 'Earnings_Score']
                except Exception:
                    pass

            combined_df.loc[y, 'Earnings_Score'] = earnings_score


        if not os.path.exists(os.getcwd() + '\\Earnings\\'):
            os.makedirs(os.getcwd() + '\\Earnings\\')

        combined_df.to_csv(os.getcwd() + '\\Earnings\\' + symbol + ' - '+ name +' - Earnings - 2016-2020.csv')
        print(1)

    except Exception:
        pass

    

In [ ]:
### Looping:
stocks_and_names_with_indices = pandas.read_csv('merged_NYSE_AMEX_removed_intercept_pattern.csv')

for x, y in stocks_and_names_with_indices.iterrows():
    earnings_score(symbol= y['Symbol'], name= y['Description'])